In [0]:
processName=dbutils.widgets.get('prm_processName')

nextSourceFileDateSql=f"""select max(PROCESSED_FILE_TABLE_DATE) AS NEXT_SOURCE_FILE_DATE from processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS where PROCESS_NAME = '{processName}' and PROCESS_STATUS='Completed'"""
print(nextSourceFileDateSql)
nextSourceFileDF=spark.sql(nextSourceFileDateSql)
print(nextSourceFileDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE'])

In [0]:
dbutils.widgets.help()

In [0]:
from datetime import datetime
dbutils.widgets.text("prm_dailyPricingSourceFileDate","")
print(datetime.strptime(dbutils.widgets.get("prm_dailyPricingSourceFileDate"),'%Y-%m-%d').strftime('%d%m%Y'))

In [0]:
sourceFileURL="https://retailpricing.blob.core.windows.net/labs/lab1/PW_MW_DR_01012023.csv"
bronzelayerCSVFilePath="abfss://working-labs@gpadlsudadatalakedev.dfs.core.windows.net/bronze/daily-pricing/csv"

In [0]:
dailyPricingSourceBaseURL='https://retailpricing.blob.core.windows.net/'
dailyPricingSourceFolder='daily-pricing/'
dialyPricingSourceFileDate=datetime.strptime(str(nextSourceFileDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE']),'%Y-%m-%d').strftime('%d%m%Y')
dailyPricingSourceFileName=f"PW_MW_DR_{dialyPricingSourceFileDate}.csv"
print(dailyPricingSourceFileName)

dailyPricingSinkLayerName='bronze'
dailyPricingSinkStorageAccountName='gpadlsudadatalakedev'
dailyPricingSinkFolderName='daily-pricing'

In [0]:
import pandas as pds


In [0]:
dailyPricingSourceURL=dailyPricingSourceBaseURL+dailyPricingSourceFolder+dailyPricingSourceFileName
print(dailyPricingSourceURL)

In [0]:
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceURL)
print(dailyPricingPandasDF)

In [0]:
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)

In [0]:
#display(dailyPricingSparkDF.limit(0))
#dailyPricingSparkDF.printSchema()
#dailyPricingSparkDF.columns[0]
display(dailyPricingSparkDF.schema)

# Show column names and data types in a tabular format
#cols_df = spark.createDataFrame(dailyPricingSparkDF.dtypes, ["column_name", "data_type"])
#display(cols_df)

In [0]:
dailyPricingSinkFolderPath=f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"
print(dailyPricingSinkFolderPath)

(
    dailyPricingSparkDF
    .write
    .mode("overwrite")
    .option("header","true")
    .csv(dailyPricingSinkFolderPath)
)

In [0]:
dbutils.fs.ls(dailyPricingSinkFolderPath)

In [0]:
%sql
drop table processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS

In [0]:
%sql
USE CATALOG `pricing-analytics`;

CREATE SCHEMA IF NOT EXISTS processrunlogs;

CREATE TABLE IF NOT EXISTS processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(
  PROCESS_NAME STRING,
  PROCESSED_FILE_TABLE_DATE DATE,
  PROCESS_STATUS STRING
)

In [0]:

processFileDate=dbutils.widgets.get('prm_dailyPricingSourceFileDate')
processStatus='Completed'

processInsertSql=f"""INSERT INTO processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS VALUES('{processName}','{processFileDate}','{processStatus}')"""

spark.sql(processInsertSql)

In [0]:
%sql
select * from processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS

In [0]:
%sql
select max(PROCESSED_FILE_TABLE_DATE) from processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS where PROCESS_NAME = 'dailyPricingSourceIngest' and PROCESS_STATUS='Completed'